In [8]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import pyodbc
import numpy as np
import os
from pyzbar import pyzbar
from PIL import Image, ImageDraw, ImageFont, ImageFilter
# import cv2
from aspose.barcode import barcoderecognition
from dbr import *

In [82]:
root = tk.Tk()
root.withdraw()
files = filedialog.askopenfilenames()

In [83]:
for path in files:
    img = Image.open(path).filter(ImageFilter.GaussianBlur(2))
    
    draw = ImageDraw.Draw(img)

    font = ImageFont.truetype("C:/Windows/Fonts/Calibri.ttf", size=100)

    for d in pyzbar.decode(img):
        draw.rectangle(((d.rect.left, d.rect.top), (d.rect.left + d.rect.width, d.rect.top + d.rect.height)),
                    outline=(0, 0, 255), width=3)
        draw.polygon(d.polygon, outline=(0, 255, 0), width=3)
        draw.text((d.rect.left, d.rect.top + d.rect.height), d.data.decode(),
                (255, 0, 0), font=font)
    img = img.resize((768, 768))
    img.show()
    # img.save('./barcode_qrcode_pillow.jpg')

## ASPOSE barcode detector

https://products.aspose.com/total/python-net/recognize-barcode/code39/

In [60]:
reader = barcoderecognition.BarCodeReader("./barcode3.jpg", barcoderecognition.DecodeType.CODE_39_STANDARD)
recognized_results = reader.read_bar_codes()
for barcode in recognized_results:
    print(barcode.code_text)

J0AZHY3


## Dynamsoft dbr barcode detector

https://github.com/Dynamsoft/barcode-reader-python-samples/blob/master/samples/image-decoding.py

In [13]:
# get the picture
def get_photo():
    root = tk.Tk()
    root.withdraw()
    path = filedialog.askopenfilename()
    path = "../" + path.split("/")[-2:][0] + "/" + path.split("/")[-2:][-1]
    return path

In [24]:
def detectBarcode(pic_path):
    error = BarcodeReader.init_license("t0073oQAAAFWxk2YxRpwXy62E0z2cGPuWUca1mhC6dbsiqIzy3i5ropGhN9cEE7uRc2jjOKncUzrtZrpquVYLTT7SZ15FwlneArgjAg==")
    if error[0] != EnumErrorCode.DBR_OK:
        print("License error: "+ error[1])
    dbr = BarcodeReader()
    image_path = pic_path
    result = dbr.decode_file(image_path)
    for res in result:
        if res.barcode_text:
            content = res.barcode_text
    return content

# i = 0
# for res in result:
#     barcode_format = res.barcode_format_string
#     print("Barcode " + str(i) + ":" + barcode_format + "," + res.barcode_text)
#     i += 1

In [25]:
path = get_photo()

In [26]:
detectBarcode(path)

'[Attention(exceptionCode:-20111)] J*AZ**3'

## barcode raw data整理

In [14]:
def get_path():
    root = tk.Tk()
    root.withdraw()
    path = filedialog.askopenfilename()
    return path

In [15]:
path = get_path()

In [16]:
dfs = pd.read_excel(path, None)

In [17]:
dfs.keys()

dict_keys(['INVOICE', 'INV-GCOS', 'Y2018', 'Y2019', 'Y2020', 'Y2021', 'Y2022', 'Y2023'])

In [18]:
gcos_dict = dict(zip(dfs["INV-GCOS"]["GCOS_No"], dfs["INV-GCOS"]["InvoiceNo"]))

In [19]:
invoice = dfs["INVOICE"]
invoice_dict = dict(zip(invoice["InvoiceNo"], list(zip(invoice["LoadingDate"], invoice["ETA"]))))

In [20]:
result = pd.concat([dfs[i] for i in dfs.keys() if "Y" in i])

In [21]:
result["GCOS_NO"] = result["GCOS_NO"].str.strip()

In [22]:
result["GT_CODE"] = result["GT_CODE"].str.strip()

In [23]:
result["INVOICE_NO"] = result["GCOS_NO"].map(gcos_dict)

In [24]:
result["INVOICE_INFO"] = result["INVOICE_NO"].map(invoice_dict)

In [25]:
result[["LOADING_DATE", "ETA"]] = pd.DataFrame(result["INVOICE_INFO"].tolist(), index=result.index, columns=["LOADING_DATE", "ETA"])

In [26]:
result = result.astype({"SAVE_DATE": "datetime64[D]", "ETA": "datetime64[D]"})

In [27]:
result = result[['SAVE_DATE', 'GCOS_NO', 'GT_CODE', 'BARCODE', 'SERIAL', 'YEAR',
       'INVOICE_NO', 'LOADING_DATE', 'ETA']]

In [29]:
path = get_path()

In [33]:
ref_data = pd.read_excel(path, header=1)

In [36]:
ref_data = ref_data[ref_data["P.Code"].notna()]

In [44]:
ref_data = ref_data.astype({"P.Code": str, "Code": str, "Section": str, "Rim": str, "Pattern": str})

In [49]:
ref_data = ref_data[['P.Code', 'Code', 'Section', 'Rim', 'Pattern']]

In [55]:
for column in ref_data.columns:
    ref_data[column] = ref_data[column].map(lambda x: x.strip())

C:\Users\kc.hsu\AppData\Local\Temp\ipykernel_16244\1245005433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_data[column] = ref_data[column].map(lambda x: x.strip())


In [58]:
ref_data["Size"] = ref_data["Section"] + ref_data["Rim"]

In [63]:
ref_data["values"] = ref_data[["Code", "Size", "Pattern"]].apply(tuple, axis=1)

In [66]:
gt_code_dict = dict(zip(ref_data["P.Code"], ref_data["values"]))

In [68]:
result["GT_CODE"] = result["GT_CODE"].map(lambda x: x.strip())

In [71]:
result["Infos"] = result["GT_CODE"].map(gt_code_dict)

In [75]:
result[["PCODE", "Size", "Pattern"]] = pd.DataFrame(result["Infos"].tolist(), index=result.index, columns=["PCODE", "Size", "Pattern"])

In [78]:
result = result[['SAVE_DATE', 'GCOS_NO', 'GT_CODE', 'BARCODE', 'SERIAL', 'YEAR',
       'INVOICE_NO', 'LOADING_DATE', 'ETA', 'PCODE', 'Size',
       'Pattern']]

In [79]:
result

,SAVE_DATE,GCOS_NO,GT_CODE,BARCODE,SERIAL,YEAR,INVOICE_NO,LOADING_DATE,ETA,PCODE,Size,Pattern
0,2018-08-04,GC1800007051,EG05,JHRUVTW,28,2018,8T8A09,2018-08-05,2018-08-19,TBR0J085,12R22.5,R224FZ
1,2018-08-04,GC1800007051,EG05,JHRUVT2,28,2018,8T8A09,2018-08-05,2018-08-19,TBR0J085,12R22.5,R224FZ
2,2018-08-04,GC1800007051,EG05,JHRUVTX,28,2018,8T8A09,2018-08-05,2018-08-19,TBR0J085,12R22.5,R224FZ
3,2018-08-04,GC1800007051,EG05,JHRUVTY,28,2018,8T8A09,2018-08-05,2018-08-19,TBR0J085,12R22.5,R224FZ
4,2018-08-04,GC1800007051,EG05,JHRUVTU,28,2018,8T8A09,2018-08-05,2018-08-19,TBR0J085,12R22.5,R224FZ
...,...,...,...,...,...,...,...,...,...,...,...,...
25456,2023-05-21,GC2300005618,FM32,J7XW2WS,20,2023,3T5B3V,NaT,NaT,TBR0J795,315/80R22.5,R150Z
25457,2023-05-21,GC2300005618,FM32,J7XW2Y3,20,2023,3T5B3V,NaT,NaT,TBR0J795,315/80R22.5,R150Z
25458,2023-05-21,GC2300005618,FM32,J7XW2XV,20,2023,3T5B3V,NaT,NaT,TBR0J795,315/80R22.5,R150Z
25459,2023-05-21,GC2300005618,FM32,J7XW2Y9,20,2023,3T5B3V,NaT,NaT,TBR0J795,315/80R22.5,R150Z


In [199]:
def menu():
    os.system("cls")
    # os.system('clear')
    print()
    print("水貨查詢小幫手")
    print("---------------------------")
    print("1. 條碼檢查和履歷")
    print("2. 資料匯出")
    print("0. 結束程式")
    print("---------------------------")

In [200]:
def pi_detection(res):
    if res.empty is True:
        print("這條是水貨，請選購有保障的公司貨")
    else:
        print("普利司通公司貨無誤~~~")

In [206]:
def check_bcode():
    pi_list = []
    while True:
        # bcode = str(input("請輸入要查詢的BarCode:\n"))
        # filter = (result["BARCODE"] == bcode.upper().strip())
        path = get_photo()
        barcode_text = detectBarcode(path)
        filter = (result["BARCODE"] == barcode_text.upper().strip())
        response = result[filter]
        pi_detection(response)
        try:
            print(response.iloc[0, :])
        except:
            print("無輪胎履歷可以提供")
            # pi_list.append(bcode)
            pi_list.append(barcode_text)
        finally:
            os.system("cls")
            try:
                ans = eval(input("要查下一條請按[1]，結束請按任意鍵\n"))
            except:
                print("謝謝使用")
                break
            else:
                if ans == 1:
                    os.system("cls")
                    continue
                else:
                    print("謝謝使用")
                    break
    return pi_list


In [210]:
df = None
while True:
    menu()
    ans = int(input(("請輸入您的選擇：\n")))
    if ans == 1:
        df = check_bcode()
    elif ans == 2:
        print("資料輸出")
        print(pd.DataFrame({"水貨BARCODE": df}))
        break
    else:
        print("謝謝使用")
        break

    
    
    


水貨查詢小幫手
---------------------------
1. 條碼檢查和履歷
2. 資料匯出
0. 結束程式
---------------------------
../PI project/liang5.jpg
普利司通公司貨無誤~~~
SAVE_DATE       2018-11-19 00:00:00
GCOS_NO                GC1800011152
GT_CODE                        NK28
BARCODE                     J15D4Y6
SERIAL                           45
YEAR                           2018
INVOICE_NO                   8TYA31
LOADING_DATE    2018-11-17 00:00:00
ETA             2018-12-02 00:00:00
Name: 29800, dtype: object
../PI project/liang6.jpg


TypeError: 'NoneType' object is not iterable

In [174]:
img = cv2.imread(r"C:\Users\kc.hsu\Downloads\barcode.png")
barcode = pyzbar.decode(img)

In [185]:
for b in barcode:
    (x, y, w, h) = b.rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (255,0,0), 5)
    print(b.data)
    print(b.type)

b'15DU8R'
CODE39


In [2]:
server = "10.226.0.53"
database = "SAP_DW"
username = "BI_User"
password = "BsSQLDB"
driver='SQL Server'

In [3]:
cnxn = pyodbc.connect(driver=driver, server=server, database=database, uid=username, password=password)

In [4]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230101'"

In [5]:
df = pd.read_sql(script, cnxn)

c:\Users\kc.hsu\Python workspace\python_env\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
